# Assignment 1

**Group 22**

A. Siganos ( 1283871 )     
J. Gómez Robles ( 1286552 )

In [82]:
import numpy as np
np.random.seed(13) #TODO Check if this is used for sgd
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Lambda
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot
from keras.preprocessing import sequence
from gensim.models import KeyedVectors
from sklearn.manifold import TSNE
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors as nn
from matplotlib import pylab
from __future__ import division

In [83]:
# DO NOT Modify the lines in this cell
path = 'alice.txt'
corpus = open(path).readlines()[0:700]

corpus = [sentence for sentence in corpus if sentence.count(" ") >= 2]

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'+"'")
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
V = len(tokenizer.word_index) + 1

# Is this something they need to change?
dim = 100
window_size = 2 #use this window size for Skipgram, CBOW, and the model with the additional hidden layer
window_size_corpus = 4 #use this window size for the co-occurrence matrix

## Question 1

### Co-occurrence Matrix
Use the provided code to load the "Alice in Wonderland" text document. 
1. Implement the word-word co-occurrence matrix for “Alice in Wonderland”
2. Normalize the words such that every value lies within a range of 0 and 1
3. Compute the cosine distance between the given words:
    - Alice 
    - Dinah
    - Rabbit
4. List the 5 closest words to 'Alice'. Discuss the results.
5. Discuss what the main drawbacks are of a term-term co-occurence matrix solutions?


In [65]:
################################################################################
############################## 0. Util functions ###############################
################################################################################

# Run this block first
def getKeyFromValue(dictionary, target):
    return list(dictionary.keys())[list(dictionary.values()).index(target)]


In [66]:
################################################################################
################ 1a. Computing word-word co-ocurrence matrix ###################
################################################################################

# Not changing V in the cell they do not want us to modify, but one never uses a 
# size variable to handle an iteration issue.
#V = len(tokenizer.word_index)

# Create my co-ocurrence matrix, initially 0 (VxV size)
# How to access this matrix:
#   Each column is an (index - 1) (column 0 is word in index 1; column 1 is word in index 2; ...)
wcoMatrix = np.zeros([V, V]) 

# Using my window_size_corpus to define my context scope
scope = window_size_corpus

# TODO: Do not make that many iterations
# Greedy approach first (to be able to compare the optimization)

# For each line in the corpus. Note that they preserve the order, even when they are indexes now.
for s in corpus:
    # For each word
    for current_index in range(0, len(s)):
        current_value = s[current_index] # Get the 'word'
        # From left to right
        for neighbor_index in range(current_index - scope, current_index + scope + 1):
            # Never out of boundaries and never the same index
            if ( neighbor_index >= 0 and neighbor_index < len(s) ) and ( neighbor_index != current_index ):
                # Get my neighbor 'word'
                neighbor = s[neighbor_index] 
                # Never myself and myself (keeping diagonal to 0)
                if current_value != neighbor:
                    # Update the ocurrence
                    #wcoMatrix[current_value - 1, neighbor - 1] += 1
                    wcoMatrix[current_value, neighbor] += 1
                    
# Finally normalizing
wcoMatrix = wcoMatrix / wcoMatrix.max()
wcoMatrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.93421053, ..., 0.01315789, 0.        ,
        0.        ],
       [0.        , 0.93421053, 0.        , ..., 0.01315789, 0.01315789,
        0.01315789],
       ...,
       [0.        , 0.01315789, 0.01315789, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.01315789, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.01315789, ..., 0.        , 0.        ,
        0.        ]])

In [67]:
################################################################################
############ 1b. Find cosine similarity to Alice, Dinah and Rabbit #############
################################################################################

words_to_compare = ["Alice", "Dinah", "Rabbit"]

# Get my index
full_index = tokenizer.word_index

# Iterate on all possible combinations
for w1 in words_to_compare:
    w1 = w1.lower() 
    x = full_index[w1]
    for w2 in words_to_compare:
        w2 = w2.lower()
        if w1 != w2:
            y = full_index[w2]
            X = wcoMatrix[x, :].reshape((1, V))
            Y = wcoMatrix[y, :].reshape((1, V))
            print("cosine_similarity(%s, %s)=%s" % (w1, w2, cosine_similarity(X, Y)))

cosine_similarity(alice, dinah)=[[0.39360011]]
cosine_similarity(alice, rabbit)=[[0.47890931]]
cosine_similarity(dinah, alice)=[[0.39360011]]
cosine_similarity(dinah, rabbit)=[[0.29862324]]
cosine_similarity(rabbit, alice)=[[0.47890931]]
cosine_similarity(rabbit, dinah)=[[0.29862324]]


In [68]:
################################################################################
##################### 1c. Find the closest words to Alice #######################
################################################################################

# It was not importing
from sklearn.neighbors import NearestNeighbors

# Define our variables
targetWord = "Alice".lower()
X = wcoMatrix
N_NEIGHBORS = 6
full_index = tokenizer.word_index

# Perfrom the NearestNeibhbors algorithm
# TODO: Tune parameters
nbrs = NearestNeighbors(n_neighbors = N_NEIGHBORS).fit(X)

# Get the index corresponding to the target word
aliceMatrixIndex = full_index[targetWord]

# Get results from nearest neighbors
distances, indices = nbrs.kneighbors()
aliceIndices = indices[aliceMatrixIndex, :]
aliceDistances = distances[aliceMatrixIndex, :]

# Output result
print("Nearest neighbors (co-ocurrence matrix based) to %s (index %s):" % (targetWord, aliceMatrixIndex))
for i in range(1, N_NEIGHBORS):
    print( "%s [index %s] with distance: %s" % (getKeyFromValue(full_index, aliceIndices[i]), aliceIndices[i], aliceDistances[i]) )

# Note: Not sure if the co-ocurrence matrix is correct or we should create a matrix with the cosine similarity
#       between each pair of words (that is the real distance between words, isn't?)

# # Create overall cosine similarity matrix
# cosSimMatrix = np.zeros([V, V]) 
# for k1, v1 in enumerate(full_index):
#     x = full_index[v1]
#     for k2, v2 in enumerate(full_index):
#         if v1 != v2:
#             y = full_index[v2]
#             X = wcoMatrix[x, :].reshape((1, V))
#             Y = wcoMatrix[y, :].reshape((1, V))

# X = cosSimMatrix
# nbrs = NearestNeighbors(n_neighbors = N_NEIGHBORS + 1).fit(X)

# # Get results from nearest neighbors
# distances, indices = nbrs.kneighbors()
# aliceIndices = indices[aliceMatrixIndex, :]
# aliceDistances = distances[aliceMatrixIndex, :]

# # Output result
# print("\nNearest neighbors (cosine similarity matrix based) to %s (index %s):" % (targetWord, aliceMatrixIndex))
# for i in range(1, N_NEIGHBORS):
#     if aliceIndices[i] != 0:
#         print( "%s [index %s] with distance: %s" % (getKeyFromValue(full_index, aliceIndices[i]), aliceIndices[i], aliceDistances[i]) )

# TODO: Think about what we are computing. 
# With first approach, a neighbor would be a word with a similar vector in the co-ocurrence matrix, 
# not a word within the same context (window_size). Although words in the similar window_size should have a
# similar vector, isn't?

Nearest neighbors (co-ocurrence matrix based) to alice (index 11):
that [index 13] with distance: 0.5692975698101006
herself [index 41] with distance: 0.5700573426956416
for [index 18] with distance: 0.5811858638252018
on [index 21] with distance: 0.5837124477057559
be [index 20] with distance: 0.5890270864631393


**Discussion of the drawbacks**

TODO

In [69]:
#Save your all the vector representations of your word embeddings in this way
#Change when necessary the sizes of the vocabulary/embedding dimension

f = open('vectors_co_occurrence.txt',"w")
f.write(" ".join([str(V-1),str(V-1)]))
f.write("\n")

#vectors = your word co-occurrence matrix
vectors = wcoMatrix
for word, i in tokenizer.word_index.items():    
    f.write(word)
    f.write(" ")
    f.write(" ".join(map(str, list(vectors[i,:]))))
    f.write("\n")
f.close()

In [70]:
#reopen your file as follows
# TODO: Check! It is not reloading the file D:
co_occurrence = KeyedVectors.load_word2vec_format('./vectors_co_occurrence.txt', binary=False)

ValueError: invalid vector on line 0 (is this really the text format?)

## Question 2

### Word embeddings
Build embeddings with a keras implementation where the embedding vector is of length 50, 150 and 300. Use the Alice in Wonderland text book for training.
1. Using the CBOW model
2. Using Skipgram model
3. Add extra hidden dense layer to CBow and Skipgram implementations. Choose an activation function for that layer and justify your answer.
4. Analyze the four different word embeddings
    - Implement your own function to perform the analogy task with. Do not use existing libraries for this task such as Gensim. Your function should be able to answer whether an anaology as in the example given in the pdf-file is true.
    - Compare the performance on the analogy task between the word embeddings that you have trained in 2.1, 2.2 and 2.3.  
    - Visualize your results and interpret your results
5. Use the word co-occurence matrix from Question 1. Compare the performance on the analogy task with the performance of your trained word embeddings.  
6. Discuss:
    - What are the main advantages of CBOW and Skipgram?
    - What is the advantage of negative sampling?
    - What are the main drawbacks of CBOW and Skipgram?
7. Load pre-trained embeddings on large corpuses (see the pdf file). You only have to consider the word embeddings with an embedding size of 300
    - Compare performance on the analogy task with your own trained embeddings from "Alice in Wonderland". You can limit yourself to the vocabulary of Alice in Wonderland. Visualize the pre-trained word embeddings and compare these with the results of your own trained word embeddings. 


In [ ]:
#prepare data for cbow


In [ ]:
#create CBOW model


In [ ]:
#define loss function


In [ ]:
#train model


In [149]:
#prepare data for Skipgram

# This function creates a list where each word in the text file (with non-empty content)
# is encoded into two arrays:
#   - x: an array of the size of all his windows_size neighbors with the value of the word(i.e, repeated values)
#   - y: an array of the same size as x pointing to each neighbor. Finally encoded in a |V|-list as one-hot enc
def generate_data_skipgram(corpus, window_size, V):
    all_in = [] 
    all_out = []
    for sentence in corpus: # For, sentence in corpus
        L = len(sentence) # Limit for the iteration (per sentence)

        # For each pair in the index
        for index, word in enumerate(sentence):
            # Create X and Y
            in_words = [] # list of lists
            labels = [] # list of integers

            # Iterate over all the sentence, for each word in the line. I.e, for each word:
            for i in range(index - window_size, index + window_size + 1):
                # If between the limits and not myself
                if i != index and (0 <= i < L):
                    # This means: Associating the words in my neighbor (context) 
                    in_words.append([word]) # Current word (as a list) is associated to...
                    labels.append(sentence[i]) # ... this word (as a simple integer).
                    
            # Once I have saved the context for the current word...
            # ... if there is something in the arrays
            if in_words != []:
                # Save the information to the all_in/out arrays
                all_in.append(np.array(in_words,dtype=np.int32)) # X in integer format
                # Set Y to categorical values. If my output is ([6, 26]). Then I create
                # a vector from 0 to V and mark 6 and 26 as 1
                all_out.append(np_utils.to_categorical(labels, V))
    return (all_in,all_out) # return the values

# Prepare data for skipgram
X, Y = generate_data_skipgram(corpus, window_size, V)


In [150]:
# #save the preprocessed data of Skipgram
# f = open('data_skipgram.txt' ,'w')

# for input, outcome in zip(X,Y):
#     input = np.concatenate(input)
#     f.write(" ".join(map(str, list(input))))
#     f.write(",")
#     outcome = np.concatenate(outcome)
#     f.write(" ".join(map(str,list(outcome))))
#     f.write("\n")
# f.close()


In [151]:
# #load the preprocessed Skipgram data
# def generate_data_skipgram_from_file():
#     f = open('data_skipgram.txt' ,'r')
#     for row in f:
#         inputs,outputs = row.split(",")
#         inputs = np.fromstring(inputs, dtype=int, sep=' ')
#         inputs = np.asarray(np.split(inputs, len(inputs)))
#         outputs = np.fromstring(outputs, dtype=float, sep=' ')
#         outputs = np.asarray(np.split(outputs, len(inputs)))
#         yield (inputs,outputs)
        

In [152]:
# create Skipgram model

# The Sequential model is a linear stack of layers.

skipgram = Sequential()
skipgram.add(Embedding(input_dim = V, output_dim = dim, embeddings_initializer = 'glorot_uniform', input_length = 1))
skipgram.add(Reshape((dim, )))
skipgram.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))


In [153]:
#define loss function for Skipgram
skipgram.compile(loss='categorical_crossentropy', optimizer='adadelta')


In [154]:
#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in zip(X, Y):
        loss += skipgram.train_on_batch(x, y)

    print(ite, loss)
    

ValueError: Error when checking target: expected reshape_8 to have shape (100,) but got array with shape (1183,)

In [ ]:
#create CBOW model with additional dense layer


In [ ]:
#define loss function for CBOW + dense


In [ ]:
#train model for CBOW + dense


In [155]:
#create Skipgram with additional dense layer

skipgram = Sequential()
skipgram.add(Embedding(input_dim = V, output_dim = dim, embeddings_initializer = 'glorot_uniform', input_length = 1))
skipgram.add(Reshape((dim, )))
skipgram.add(Dense(input_dim=dim, units=V, kernel_initializer='uniform', activation='softmax'))



In [156]:
#define loss function for Skipgram + dense
skipgram.compile(loss='categorical_crossentropy', optimizer='adadelta')


In [157]:
#train model for Skipgram + dense
#train Skipgram model
for ite in range(10):
    loss = 0.
    for x, y in zip(X, Y):
        loss += skipgram.train_on_batch(x, y)

    print(ite, loss)



0 41242.59084939957
1 38998.47589588165
2 39104.93803155422
3 39152.549574136734
4 39207.58228170872
5 39277.39926147461
6 39361.115528941154
7 39453.87337732315
8 39547.44167852402
9 39634.98483276367


In [ ]:
#Implement your own analogy function



Comparison performance:

In [ ]:
#Visualization results trained word embeddings


Interpretation results of the visualization

Compare the results of the trained word embeddings with the word-word co-occurrence matrix

Discussion of the advantages of CBOW and Skipgram, the advantages of negative sampling and drawbacks of CBOW and Skipgram

In [ ]:
#load pretrained word embeddings of word2vec

path_word2vec = "your path /GoogleNews-vectors-negative300.bin"

word2vec = KeyedVectors.load_word2vec_format(path, binary=True)

In [ ]:
#load pretraind word embeddings of Glove

path = "your path /glove.6B/glove.6B.300d_converted.txt"

#convert GloVe into word2vec format
gensim.scripts.glove2word2vec.get_glove_info(path)
gensim.scripts.glove2word2vec.glove2word2vec(path, "glove_converted.txt")

glove = KeyedVectors.load_word2vec_format(path, binary=False)

In [ ]:
#Visualize the pre-trained word embeddings

Comparison performance with your own trained word embeddings